<a href="https://colab.research.google.com/github/crismunoz/Fuzzy-Logic/blob/master/Extra%C3%A7%C3%A3o_de_Regras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-fuzzy

In [ ]:
import tensorflow as tf
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from fuzzy_rules import extract_rules,config_input_variable,config_output_variable
from sklearn.metrics import mean_squared_error

In [ ]:
file_path='Dolar.txt'
windows_size=5
nb_sets = 5

def preprocessing(line):
  line = line.strip().replace(',','.')
  return float(line)

with open(file_path,'r') as file:
  series = [preprocessing(line) for line in file]

dataset = tf.data.Dataset.from_tensor_slices(series)
dataset = dataset.window(windows_size + 1, shift=1, drop_remainder=True)

data = np.stack([list(window_dataset) for window_dataset in dataset], axis=0)
print('data.shape: {}'.format(data.shape))
min_value = data.min()
max_value = data.max()

print('File Path: {}\nWindows Siz: {}\nMin Value: {:.2f}\nMax Value: {:.2f}'.\
      format(file_path, windows_size, min_value, max_value))

In [ ]:
var_config = {
'windows_size':windows_size,
'nb_sets':nb_sets,
'min':min_value-0.01,
'max':max_value+0.01,
'type':'input & output'
}

In [ ]:
input_variables  = [config_input_variable('i_{}'.format(i+1), var_config['nb_sets'], var_config['min'], var_config['max']) for i in range(var_config['windows_size'])]
output_variables = [config_output_variable('output', var_config['nb_sets'], var_config['min'], var_config['max'])]
variables = input_variables + output_variables

In [ ]:
rules = extract_rules(variables, data)
system = ctrl.ControlSystem(rules)
sim = ctrl.ControlSystemSimulation(system)

In [ ]:
output_real = []
output_prev = []
for instance in data:
  for i,v in enumerate(instance[:-1]):
    sim.input['i_{}'.format(i+1)]=instance[i]
  sim.compute()
  output_prev.append(sim.output['output'])
  output_real.append(instance[-1])

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(output_real,'*')
plt.plot(output_prev,'s')

In [ ]:
mean_squared_error(output_real, output_prev)